In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import json, os, cv2, time
from tqdm import tqdm

# ===================================================
# CONFIG
# ===================================================
DATA_PATH = "D:/Semester/Semester5/DPL302/Project/sentence_dataset/"
METADATA_PATH = 'D:/Semester/Semester5/DPL302/Project/sentence_dataset/metadata.json'
BATCH_SIZE = 8
EPOCHS = 10
LR = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_SEQ_LEN = 60  # chuẩn hóa tạm thời
TRAIN_RATIO, VAL_RATIO = 0.8, 0.1



In [11]:
# ===================================================
# DATASET
# ===================================================
class GestureCTCDataset(Dataset):
    def __init__(self, data_dir, metadata_path):
        self.data_dir = data_dir
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        all_labels = sorted({lbl for item in self.metadata for lbl in item["labels"]})
        self.label_to_idx = {lbl: i + 1 for i, lbl in enumerate(all_labels)}  # 0 là blank
        self.idx_to_label = {i + 1: lbl for i, lbl in enumerate(all_labels)}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        item = self.metadata[idx]
        keypoints = np.load(os.path.join(self.data_dir, item["file"]))  # (T,2,21,2)
        keypoints = keypoints.reshape(keypoints.shape[0], -1)  # (T,84)
        label_seq = [self.label_to_idx[lbl] for lbl in item["labels"]]
        return torch.tensor(keypoints, dtype=torch.float32), torch.tensor(label_seq, dtype=torch.long)

def collate_fn(batch):
    inputs, labels = zip(*batch)
    input_lengths = torch.tensor([len(x) for x in inputs])
    label_lengths = torch.tensor([len(l) for l in labels])
    inputs_padded = pad_sequence(inputs, batch_first=True)
    labels_concat = torch.cat(labels)
    return inputs_padded, labels_concat, input_lengths, label_lengths



In [12]:
# ===================================================
# MODEL: Transformer Encoder + CTC
# ===================================================
class TransformerCTC(nn.Module):
    def __init__(self, input_dim, model_dim, num_classes, num_layers=4, num_heads=8, ff_dim=512):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=0.2,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.encoder(x)
        return self.output_fc(x)



In [13]:
# ===================================================
# LOAD DATA
# ===================================================
dataset = GestureCTCDataset(DATA_PATH, METADATA_PATH)
train_size = int(TRAIN_RATIO * len(dataset))
val_size = int(VAL_RATIO * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, collate_fn=collate_fn)

num_classes = len(dataset.label_to_idx) + 1
input_dim = 84



In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import mediapipe as mp
from collections import deque, Counter
import json

# ===================================================
# MODEL (GIỐNG TRAINING)
# ===================================================
class TransformerCTC(nn.Module):
    def __init__(self, input_dim, model_dim, num_classes, num_layers=2, num_heads=4, ff_dim=256):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=0.2,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.encoder(x)
        return self.output_fc(x)

# ===================================================
# CONFIG
# ===================================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = 84
MODEL_DIM = 128
MAX_SEQ_LEN = 60
DEBUG_MODE = True

MODEL_PATH = "D:/Semester/Semester5/DPL302/Project/src/transformer_ctc_gesture_best.pth"
METADATA_PATH = "D:\Semester\Semester5\DPL302\Project\sentence_dataset\metadata.json"

# ===================================================
# LOAD LABEL MAPPING
# ===================================================
def load_label_mapping(metadata_path):
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        all_labels = sorted({lbl for item in metadata for lbl in item["labels"]})
        label_to_idx = {lbl: i + 1 for i, lbl in enumerate(all_labels)}
        idx_to_label = {i + 1: lbl for i, lbl in enumerate(all_labels)}
        idx_to_label[0] = "BLANK"
        print(f"✅ Loaded {len(all_labels)} gesture classes")
        return idx_to_label, len(all_labels) + 1
    except FileNotFoundError:
        print("⚠️ Metadata not found, using default labels")
        idx_to_label = {0:"BLANK", 1:"Hello",2:"Thank_you",3:"I_love_you",4:"Yes",
                        5:"No",6:"Please",7:"Sorry",8:"Help",9:"Eat",
                        10:"Drink",11:"Good",12:"Bad",13:"More",14:"Less",
                        15:"Stop",16:"Go",17:"Come",18:"Wait",19:"Understand",
                        20:"Not_understand",21:"What",22:"Where",23:"When",24:"Who",
                        25:"How",26:"Why",27:"Can",28:"Cannot",29:"Want",30:"Not_want"}
        return idx_to_label, 31

GESTURE_LABELS, NUM_CLASSES = load_label_mapping(METADATA_PATH)

# ===================================================
# LOAD MODEL
# ===================================================
model = TransformerCTC(INPUT_DIM, MODEL_DIM, NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE), strict=False)
model.eval()
print(f"✅ Model loaded on {DEVICE}")

# ===================================================
# MEDIAPIPE SETUP
# ===================================================
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pose_detector = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# ===================================================
# KEYPOINT EXTRACTION & NORMALIZATION
# ===================================================
def get_neck_and_shoulder(frame):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_detector.process(rgb)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        shoulder_right = np.array([landmarks[11].x, landmarks[11].y])
        shoulder_left = np.array([landmarks[12].x, landmarks[12].y])
        neck = (shoulder_left + shoulder_right) / 2
        shoulder_dist = np.linalg.norm(shoulder_left - shoulder_right)
        return neck, shoulder_dist
    return None, None

def extract_hand_keypoints(res_hands):
    keypoints = np.zeros((2,21,2))
    if res_hands.multi_hand_landmarks and res_hands.multi_handedness:
        for hand_landmarks, handedness in zip(res_hands.multi_hand_landmarks, res_hands.multi_handedness):
            hand_kp = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])
            idx = 0 if handedness.classification[0].label=="Left" else 1
            keypoints[idx] = hand_kp
    return keypoints

def normalize_keypoints(keypoints, neck_point, shoulder_dist):
    normalized = np.zeros_like(keypoints)
    if neck_point is None or shoulder_dist is None or shoulder_dist < 0.01:
        return keypoints
    for i in range(2):
        hand = keypoints[i]
        if np.all(hand==0):
            continue
        normalized[i] = (hand - neck_point)/shoulder_dist
    return normalized

def prepare_input_like_training(keypoints):
    return keypoints

# ===================================================
# CTC DECODING
# ===================================================
def ctc_decode_detailed(logits):
    log_probs = F.log_softmax(logits, dim=-1)
    probs = torch.exp(log_probs)
    pred_classes = torch.argmax(log_probs, dim=-1)[0].cpu().numpy()
    max_probs = torch.max(probs, dim=-1)[0][0].cpu().numpy()
    class_counts = Counter(pred_classes)
    collapsed = []
    prev=-1
    for cls in pred_classes:
        if cls!=0 and cls!=prev:
            collapsed.append(cls)
        prev = cls
    return {
        'raw_predictions': pred_classes,
        'max_probs': max_probs,
        'class_counts': class_counts,
        'collapsed': collapsed,
        'avg_confidence': np.mean(max_probs),
        'non_blank_ratio': 1-(class_counts.get(0,0)/len(pred_classes))
    }

# ===================================================
# DEBUG PANEL
# ===================================================
def draw_debug_panel(frame, info_dict, y_start=100):
    h,w=frame.shape[:2]
    panel_height = len(info_dict)*25+20
    cv2.rectangle(frame,(10,y_start),(w-10,y_start+panel_height),(0,0,0),-1)
    cv2.rectangle(frame,(10,y_start),(w-10,y_start+panel_height),(255,255,0),2)
    y_offset=y_start+25
    for key,value in info_dict.items():
        text=f"{key}: {value}"
        cv2.putText(frame,text,(20,y_offset),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),1)
        y_offset+=25

# ===================================================
# MAIN LOOP
# ===================================================
def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Cannot open camera"); return

    # Calibration
    print("🔧 Calibrating pose... Stand still")
    neck_samples=[]
    shoulder_samples=[]
    for i in range(100):
        ret, frame = cap.read()
        if not ret: continue
        neck, dist = get_neck_and_shoulder(frame)
        if neck is not None:
            neck_samples.append(neck)
            shoulder_samples.append(dist)
        if (i+1)%20==0: print(f"Progress: {i+1}/100")
    avg_neck=np.mean(neck_samples, axis=0) if neck_samples else np.array([0.5,0.5])
    avg_shoulder=np.mean(shoulder_samples) if shoulder_samples else 0.2
    print(f"✅ Calibration done: neck={avg_neck}, shoulder_dist={avg_shoulder}")

    seq_buffer = deque(maxlen=MAX_SEQ_LEN)
    gesture_history = deque(maxlen=10)
    current_gesture="Waiting..."
    current_confidence=0.0
    frame_count=0

    while True:
        ret, frame = cap.read()
        if not ret: break
        frame=cv2.flip(frame,1)
        h,w=frame.shape[:2]
        rgb=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res_hands=hands_detector.process(rgb)

        keypoints_raw=extract_hand_keypoints(res_hands)
        keypoints_norm=normalize_keypoints(keypoints_raw, avg_neck, avg_shoulder)
        input_data=prepare_input_like_training(keypoints_norm)
        seq_buffer.append(input_data)

        input_flat=input_data.reshape(-1)
        has_left=not np.all(keypoints_raw[0]==0)
        has_right=not np.all(keypoints_raw[1]==0)
        debug_info={
            'Buffer': f"{len(seq_buffer)}/{MAX_SEQ_LEN}",
            'Hands': f"L:{has_left} R:{has_right}",
            'Input Stats': f"mean={input_flat.mean():.3f} std={input_flat.std():.3f}",
            'Input Range': f"[{input_flat.min():.3f}, {input_flat.max():.3f}]"
        }

        # Predict khi buffer đầy
        if len(seq_buffer) == MAX_SEQ_LEN and frame_count % 3 == 0:
            seq_array = np.array(seq_buffer)             # (T, 2, 21, 2)
            seq_flat = seq_array.reshape(seq_array.shape[0], -1)  # (T, 84)
            x = torch.tensor(seq_flat, dtype=torch.float32).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                logits = model(x)  # (1, T, NUM_CLASSES)
                decode_result = ctc_decode_detailed(logits)

            # Debug console
            if DEBUG_MODE:
                print(f"\n{'='*40}")
                print(f"Frame {frame_count}")
                print(f"Top 5 classes: {decode_result['class_counts'].most_common(5)}")
                print(f"Collapsed: {decode_result['collapsed']}")
                print(f"Avg confidence: {decode_result['avg_confidence']:.3f}")
                print(f"Non-blank ratio: {decode_result['non_blank_ratio']:.3f}")

            debug_info['Top Classes'] = str(decode_result['class_counts'].most_common(3))
            debug_info['Non-blank%'] = f"{decode_result['non_blank_ratio']*100:.1f}%"
            debug_info['Avg Conf'] = f"{decode_result['avg_confidence']:.3f}"

            collapsed = decode_result['collapsed']
            if collapsed:
                most_common_cls = Counter(collapsed).most_common(1)[0][0]
                gesture_history.append(most_common_cls)
                if len(gesture_history) >= 3:
                    smoothed_cls = Counter(gesture_history).most_common(1)[0][0]
                    current_gesture = GESTURE_LABELS.get(smoothed_cls, f"Unknown_{smoothed_cls}")
                    current_confidence = decode_result['avg_confidence']
            else:
                if DEBUG_MODE:
                    print("⚠️ All predictions are BLANK")

        # Vẽ UI
        cv2.rectangle(frame, (10,10), (w-10,90), (0,0,0), -1)
        cv2.rectangle(frame, (10,10), (w-10,90), (0,255,0), 3)
        cv2.putText(frame, current_gesture, (20,60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)

        if current_confidence > 0:
            bar_width = int((w-40) * current_confidence)
            cv2.rectangle(frame, (20,70), (20+bar_width,80), (0,255,255), -1)

        if DEBUG_MODE:
            draw_debug_panel(frame, debug_info, y_start=110)

        cv2.imshow("Gesture Recognition - Transformer CTC", frame)
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == 27:  # ESC
            break

    cap.release()
    pose_detector.close()
    hands_detector.close()
    cv2.destroyAllWindows()
    print("\n👋 Program ended")
    print("="*60)

if __name__ == "__main__":
    main()


✅ Loaded 30 gesture classes
✅ Model loaded on cpu
🔧 Calibrating pose... Stand still
Progress: 20/100
Progress: 40/100
Progress: 60/100
Progress: 80/100
Progress: 100/100
✅ Calibration done: neck=[0.50945608 0.8123345 ], shoulder_dist=0.3893596390409072

Frame 60
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 63
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 66
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 69
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 72
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 75
Top 5 classes: [(0, 60)]
Collapsed: []
Avg confidence: 0.999
Non-blank ratio: 0.000
⚠️ All predictions are BLANK

Frame 78

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

# ===================================================
# CONFIG
# ===================================================
DATA_PATH = "D:/Semester/Semester5/DPL302/Project/sentence_dataset/"
METADATA_PATH = "D:/Semester/Semester5/DPL302/Project/sentence_dataset/metadata.json"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reduced model size
MODEL_DIM = 128      # Giảm từ 256
NUM_LAYERS = 2       # Giảm từ 4
NUM_HEADS = 4        # Giảm từ 8
FF_DIM = 256         # Giảm từ 512

BATCH_SIZE = 16       # Giảm batch size
EPOCHS = 50          # Tăng epochs
LR = 1e-4            # Giảm learning rate
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15

# ===================================================
# DATASET
# ===================================================
class GestureCTCDataset(Dataset):
    def __init__(self, data_dir, metadata_path):
        self.data_dir = data_dir
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        
        all_labels = sorted({lbl for item in self.metadata for lbl in item["labels"]})
        self.label_to_idx = {lbl: i + 1 for i, lbl in enumerate(all_labels)}
        self.idx_to_label = {i + 1: lbl for i, lbl in enumerate(all_labels)}
        
        print(f"✅ Loaded {len(self.metadata)} samples")
        print(f"✅ Found {len(all_labels)} gesture classes")
        
        # Check input/label length ratio
        self._check_data_validity()
    
    def _check_data_validity(self):
        """Check if data is valid for CTC"""
        print("\n🔍 Checking data validity...")
        problematic = []
        
        for item in self.metadata:
            keypoints = np.load(os.path.join(self.data_dir, item["file"]))
            input_len = keypoints.shape[0]
            label_len = len(item["labels"])
            ratio = input_len / label_len if label_len > 0 else 0
            
            if ratio < 2:
                problematic.append((item["file"], input_len, label_len, ratio))
        
        if problematic:
            print(f"⚠️ WARNING: {len(problematic)} samples have input_length < 2 * label_length")
            print("This may cause CTC training to fail!")
            for fname, inp, lbl, r in problematic[:5]:
                print(f"  {fname}: input={inp}, label={lbl}, ratio={r:.2f}")
        else:
            print("✅ All samples have valid length ratios")
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        item = self.metadata[idx]
        keypoints = np.load(os.path.join(self.data_dir, item["file"]))
        keypoints = keypoints.reshape(keypoints.shape[0], -1)  # (T, 84)
        label_seq = [self.label_to_idx[lbl] for lbl in item["labels"]]
        return torch.tensor(keypoints, dtype=torch.float32), torch.tensor(label_seq, dtype=torch.long)

def collate_fn(batch):
    inputs, labels = zip(*batch)
    input_lengths = torch.tensor([len(x) for x in inputs])
    label_lengths = torch.tensor([len(l) for l in labels])
    inputs_padded = pad_sequence(inputs, batch_first=True)
    labels_concat = torch.cat(labels)
    return inputs_padded, labels_concat, input_lengths, label_lengths

# ===================================================
# MODEL (SMALLER)
# ===================================================
class TransformerCTC(nn.Module):
    def __init__(self, input_dim, model_dim, num_classes, num_layers=2, num_heads=4, ff_dim=256):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, model_dim)
        self.pos_encoder = PositionalEncoding(model_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=0.3,  # Tăng dropout
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_fc = nn.Linear(model_dim, num_classes)
    
    def forward(self, x):
        x = self.input_fc(x)
        x = self.pos_encoder(x)
        x = self.encoder(x)
        return self.output_fc(x)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return x + self.pe[:x.size(1), :].unsqueeze(0)

# ===================================================
# CTC LOSS WITH BLANK PENALTY
# ===================================================
class CTCLossWithPenalty(nn.Module):
    def __init__(self, blank=0, zero_infinity=True, blank_penalty=0.4):
        super().__init__()
        self.ctc_loss = nn.CTCLoss(blank=blank, zero_infinity=zero_infinity, reduction='mean')
        self.blank_penalty = blank_penalty
    
    def forward(self, log_probs, targets, input_lengths, target_lengths):
        ctc_loss = self.ctc_loss(log_probs, targets, input_lengths, target_lengths)
        
        # Penalty for predicting too many blanks
        blank_prob = torch.exp(log_probs[:, :, 0])
        blank_ratio = blank_prob.mean()
        penalty = blank_ratio * self.blank_penalty
        
        return ctc_loss + penalty, ctc_loss.item(), blank_ratio.item()

# ===================================================
# TRAINING
# ===================================================
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_ctc_loss = 0
    total_blank_ratio = 0
    
    for x, y, x_len, y_len in tqdm(loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        
        logits = model(x)
        log_probs = logits.log_softmax(2).transpose(0, 1)  # (T, B, C)
        
        loss, ctc_loss, blank_ratio = criterion(log_probs, y, x_len, y_len)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
        total_ctc_loss += ctc_loss
        total_blank_ratio += blank_ratio
    
    n = len(loader)
    return total_loss/n, total_ctc_loss/n, total_blank_ratio/n

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_ctc_loss = 0
    
    with torch.no_grad():
        for x, y, x_len, y_len in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            log_probs = logits.log_softmax(2).transpose(0, 1)
            loss, ctc_loss, _ = criterion(log_probs, y, x_len, y_len)
            total_loss += loss.item()
            total_ctc_loss += ctc_loss
    
    n = len(loader)
    return total_loss/n, total_ctc_loss/n

# ===================================================
# MAIN
# ===================================================
def main():
    print("="*60)
    print("🚀 TRAINING TRANSFORMER CTC MODEL (IMPROVED)")
    print("="*60)
    
    # Load dataset
    dataset = GestureCTCDataset(DATA_PATH, METADATA_PATH)
    num_classes = len(dataset.label_to_idx) + 1
    
    # Split
    train_size = int(TRAIN_RATIO * len(dataset))
    val_size = int(VAL_RATIO * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])
    
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    
    print(f"\n📊 Data split:")
    print(f"   Train: {train_size} samples")
    print(f"   Val: {val_size} samples")
    print(f"   Test: {test_size} samples")
    
    # Model
    model = TransformerCTC(84, MODEL_DIM, num_classes, NUM_LAYERS, NUM_HEADS, FF_DIM).to(DEVICE)
    criterion = CTCLossWithPenalty(blank=0, zero_infinity=True, blank_penalty=0.4)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    print(f"\n🔧 Model config:")
    print(f"   Model dim: {MODEL_DIM}")
    print(f"   Layers: {NUM_LAYERS}")
    print(f"   Heads: {NUM_HEADS}")
    print(f"   FF dim: {FF_DIM}")
    print(f"   Classes: {num_classes}")
    print(f"   Device: {DEVICE}")
    print(f"   Learning rate: {LR}")
    
    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    max_patience = 10
    
    print(f"\n{'='*60}")
    print("🔥 STARTING TRAINING")
    print(f"{'='*60}\n")
    
    for epoch in range(EPOCHS):
        train_loss, train_ctc, train_blank = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        val_loss, val_ctc = validate(model, val_loader, criterion, DEVICE)
        
        print(f"\n📊 Epoch {epoch+1}/{EPOCHS}")
        print(f"   Train Loss: {train_loss:.4f} | CTC: {train_ctc:.4f} | Blank: {train_blank:.2%}")
        print(f"   Val Loss: {val_loss:.4f} | CTC: {val_ctc:.4f}")
        
        # Learning rate scheduling
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        new_lr = optimizer.param_groups[0]['lr']
        if new_lr != old_lr:
            print(f"   📉 Learning rate reduced: {old_lr:.6f} → {new_lr:.6f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "transformer_ctc_gesture_best.pth")
            print(f"   ✅ Best model saved! (Val Loss: {val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= max_patience:
            print(f"\n⚠️ Early stopping triggered (no improvement for {max_patience} epochs)")
            break
    
    print(f"\n{'='*60}")
    print("✅ TRAINING COMPLETED!")
    print(f"Best Val Loss: {best_val_loss:.4f}")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()

🚀 TRAINING TRANSFORMER CTC MODEL (IMPROVED)
✅ Loaded 3000 samples
✅ Found 30 gesture classes

🔍 Checking data validity...
✅ All samples have valid length ratios

📊 Data split:
   Train: 2100 samples
   Val: 450 samples
   Test: 450 samples

🔧 Model config:
   Model dim: 128
   Layers: 2
   Heads: 4
   FF dim: 256
   Classes: 31
   Device: cpu
   Learning rate: 0.0001

🔥 STARTING TRAINING



Training: 100%|██████████| 132/132 [00:43<00:00,  3.03it/s]



📊 Epoch 1/50
   Train Loss: 23.6606 | CTC: 23.3261 | Blank: 83.61%
   Val Loss: 4.4216 | CTC: 4.0249
   ✅ Best model saved! (Val Loss: 4.4216)


Training: 100%|██████████| 132/132 [00:44<00:00,  2.94it/s]



📊 Epoch 2/50
   Train Loss: 4.1862 | CTC: 3.7920 | Blank: 98.53%
   Val Loss: 4.0888 | CTC: 3.6923
   ✅ Best model saved! (Val Loss: 4.0888)


...


📊 Epoch 48/50
   Train Loss: 0.4394 | CTC: 0.0464 | Blank: 98.26%
   Val Loss: 0.4532 | CTC: 0.0601


Training: 100%|██████████| 132/132 [00:33<00:00,  3.99it/s]



📊 Epoch 49/50
   Train Loss: 0.4428 | CTC: 0.0496 | Blank: 98.28%
   Val Loss: 0.4448 | CTC: 0.0515
   ✅ Best model saved! (Val Loss: 0.4448)


Training: 100%|██████████| 132/132 [00:32<00:00,  4.02it/s]



📊 Epoch 50/50
   Train Loss: 0.4359 | CTC: 0.0430 | Blank: 98.22%
   Val Loss: 0.4478 | CTC: 0.0546

✅ TRAINING COMPLETED!
Best Val Loss: 0.4448
